In [ ]:
import os, sys
import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import numpy as np 
import open3d as o3d


In [ ]:
print("GPU Available: ", torch.cuda.get_device_name(0))
print("Cuda version: ", torch.version.cuda)

Importing project modules

In [ ]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)


from datasets.pointnet_dataset import PointNetDataset
from models.pointnet_cls import PointNetCls

set seeds

In [ ]:
seed = 42

# Set seed for PyTorch
torch.manual_seed(seed)

# Set seed for CUDA (if using GPUs)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

# Set seed for NumPy
np.random.seed(seed)

In [ ]:
num_classes = 40
batch_size = 64

model_path = "best_pointnet.pth"

epochs=50
val_ratio = 0.15

Read and Split dataset

In [ ]:
dataset = PointNetDataset("../data/ModelNet40", split="train")

train_size = int((1-val_ratio) * len(dataset))
val_size = len(dataset) - train_size

train_data, val_data = random_split(dataset, [train_size, val_size])

Load the Dataset

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

Model and Optimizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr=0.001

In [ ]:
model = PointNetCls(num_classes=40).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)

Regularizer TNet

In [ ]:
def feature_transform_regularizer(trans_feat):

    B, k, _ = trans_feat.size()
    I = torch.eye(k, device=trans_feat.device).unsqueeze(0)
    diff = torch.bmm(trans_feat, trans_feat.transpose(2, 1)) - I
    return torch.norm(diff, dim=(1, 2)).mean()

Training Loop

In [ ]:
for epoch in range(epochs):
    
    running_loss = 0.0
    correct = 0
    total = 0

    model.train()

    for points, labels in train_loader:
        points = points.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        pred, trans_feature = model(points)

        loss_cls = criterion(pred, labels)
        loss_reg = feature_transform_regularizer(trans_feature)

        loss = loss_cls + 0.001 * loss_reg
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predicted = pred.argmax(dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    train_loss = running_loss / len(train_loader)

    # ---------- Validation ----------
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0

    with torch.no_grad():
        for points, labels in val_loader:
            points = points.to(device)
            labels = labels.to(device)
            points = points.transpose(2, 1)

            pred, _, trans_feat = model(points)
            loss_cls = criterion(pred, labels)
            loss_reg = feature_transform_regularizer(trans_feat)
            loss = loss_cls + 0.001 * loss_reg

            val_loss += loss.item()

            predicted = pred.argmax(dim=1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    val_loss = val_loss / len(val_loader)

    scheduler.step()

    print(f"Epoch [{epoch+1}/{epochs}] "
            f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% "
            f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")


Save Model

In [ ]:
torch.save(model.state_dict(), "pointnet_model.pth")
print("Model saved as pointnet_model.pth")

Testing Model

In [ ]:
test_dataset = PointNetDataset("data/ModelNet40", split="test")
test_data = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model.eval()

correct = 0
total = 0

cls_correct = np.zeros(num_classes)
cls_total = np.zeros(num_classes)

with torch.no_grad():
    for points, labels in test_data:
        points = points.to(device)
        labels = labels.to(device)

        preds, _ = model(points)
        predicted = preds.argmax(dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        for i in range(labels.size(0)):
            label = labels[i].item()
            cls_total[label] += 1
            if predicted[i].item() == label:
                cls_correct[label] += 1

In [ ]:
overall_acc = 100 * correct / total
print(f"Total Accuracy: {overall_acc}%")

print("Per-class Accuracy:")
for i in range(num_classes):
    if cls_total[i] > 0:
        acc = 100 * cls_correct[i] / cls_total[i]
        print(f"Class {i}: {acc:}%")